In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 4), abs(state[i*4+1] + state[i*4+3] * 4), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(4)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[ 1.06466167e-01  8.01375274e-02  8.41591268e-02  8.15176614e-02]
 [-1.21703288e+00 -5.12403226e-01 -2.61653441e-01 -1.06906776e-01]
 [ 2.13744683e-01  7.27000728e-02  4.91299486e-02  3.55403474e-02]
 ...
 [ 1.61184688e-03  1.90624795e-01  1.47851643e-01  4.16032662e-01]
 [-3.58954168e-01  6.76874376e-02  9.38189186e-02 -2.32618770e-01]
 [-4.90054813e-01 -1.06798833e+00 -1.80961927e+00 -1.94436221e-01]]


In [5]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    for i in range(20):
        state, reward, done, info = env.step(1)
        state, reward, done, info = env.step(3)
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action) 
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0
        
print("Training finished.\n")

Episode: 100
average reward: 44.96
Episode: 200
average reward: 13.14
Episode: 300
average reward: 25.09
Episode: 400
average reward: 67.27
Episode: 500
average reward: 58.42
Episode: 600
average reward: 83.11
Episode: 700
average reward: 38.75
Episode: 800
average reward: 40.03
Episode: 900
average reward: 45.62
Episode: 1000
average reward: -18.8
Episode: 1100
average reward: 6.73
Episode: 1200
average reward: -20.81
Episode: 1300
average reward: 27.13
Episode: 1400
average reward: 17.66
Episode: 1500
average reward: -10.06
Episode: 1600
average reward: 15.61
Episode: 1700
average reward: 70.35
Episode: 1800
average reward: 22.18
Episode: 1900
average reward: 82.18
Episode: 2000
average reward: 45.68
Episode: 2100
average reward: 10.66
Episode: 2200
average reward: 72.45
Episode: 2300
average reward: -30.61
Episode: 2400
average reward: -20.4
Episode: 2500
average reward: 61.69
Episode: 2600
average reward: 26.23
Episode: 2700
average reward: 26.76
Episode: 2800
average reward: 51.53

Episode: 22600
average reward: 50.18
Episode: 22700
average reward: 83.83
Episode: 22800
average reward: -14.1
Episode: 22900
average reward: 78.79
Episode: 23000
average reward: 66.44
Episode: 23100
average reward: -6.01
Episode: 23200
average reward: 32.93
Episode: 23300
average reward: 45.85
Episode: 23400
average reward: 111.97
Episode: 23500
average reward: 48.79
Episode: 23600
average reward: -5.71
Episode: 23700
average reward: 28.27
Episode: 23800
average reward: -14.22
Episode: 23900
average reward: 55.88
Episode: 24000
average reward: 58.03
Episode: 24100
average reward: 27.57
Episode: 24200
average reward: 23.14
Episode: 24300
average reward: 33.23
Episode: 24400
average reward: 8.12
Episode: 24500
average reward: 41.37
Episode: 24600
average reward: 11.66
Episode: 24700
average reward: -49.5
Episode: 24800
average reward: -31.14
Episode: 24900
average reward: 49.7
Episode: 25000
average reward: 58.54
Episode: 25100
average reward: 108.29
Episode: 25200
average reward: 116.8

Episode: 44800
average reward: -10.7
Episode: 44900
average reward: 22.94
Episode: 45000
average reward: 43.52
Episode: 45100
average reward: 23.31
Episode: 45200
average reward: 44.26
Episode: 45300
average reward: 19.94
Episode: 45400
average reward: -7.76
Episode: 45500
average reward: 129.15
Episode: 45600
average reward: 67.79
Episode: 45700
average reward: 63.17
Episode: 45800
average reward: 47.29
Episode: 45900
average reward: 5.52
Episode: 46000
average reward: 23.13
Episode: 46100
average reward: 4.95
Episode: 46200
average reward: 55.24
Episode: 46300
average reward: 60.54
Episode: 46400
average reward: 50.6
Episode: 46500
average reward: 25.63
Episode: 46600
average reward: -22.72
Episode: 46700
average reward: 51.92
Episode: 46800
average reward: 21.26
Episode: 46900
average reward: 68.75
Episode: 47000
average reward: 25.42
Episode: 47100
average reward: 24.57
Episode: 47200
average reward: 49.7
Episode: 47300
average reward: 1.4
Episode: 47400
average reward: -22.27
Epis

Episode: 67100
average reward: 27.56
Episode: 67200
average reward: 117.16
Episode: 67300
average reward: 106.86
Episode: 67400
average reward: 97.1
Episode: 67500
average reward: 55.44
Episode: 67600
average reward: 25.16
Episode: 67700
average reward: -49.01
Episode: 67800
average reward: 24.68
Episode: 67900
average reward: 50.32
Episode: 68000
average reward: 68.29
Episode: 68100
average reward: 6.9
Episode: 68200
average reward: 0.41
Episode: 68300
average reward: 94.27
Episode: 68400
average reward: 112.41
Episode: 68500
average reward: 31.38
Episode: 68600
average reward: 62.55
Episode: 68700
average reward: 38.96
Episode: 68800
average reward: 41.55
Episode: 68900
average reward: 15.24
Episode: 69000
average reward: 66.63
Episode: 69100
average reward: 14.95
Episode: 69200
average reward: 79.88
Episode: 69300
average reward: 28.78
Episode: 69400
average reward: 31.28
Episode: 69500
average reward: -2.49
Episode: 69600
average reward: 27.72
Episode: 69700
average reward: 37.5
Ep

Episode: 89300
average reward: 61.78
Episode: 89400
average reward: 52.66
Episode: 89500
average reward: 21.88
Episode: 89600
average reward: 59.13
Episode: 89700
average reward: -7.02
Episode: 89800
average reward: 19.31
Episode: 89900
average reward: 66.95
Episode: 90000
average reward: -39.52
Episode: 90100
average reward: 7.53
Episode: 90200
average reward: 81.5
Episode: 90300
average reward: 48.86
Episode: 90400
average reward: 60.48
Episode: 90500
average reward: 53.98
Episode: 90600
average reward: 77.38
Episode: 90700
average reward: 80.81
Episode: 90800
average reward: -16.06
Episode: 90900
average reward: 52.56
Episode: 91000
average reward: 2.87
Episode: 91100
average reward: 46.92
Episode: 91200
average reward: 87.58
Episode: 91300
average reward: 30.9
Episode: 91400
average reward: 18.57
Episode: 91500
average reward: 69.52
Episode: 91600
average reward: 41.05
Episode: 91700
average reward: 16.13
Episode: 91800
average reward: 25.79
Episode: 91900
average reward: 37.13
Epi

In [5]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [ ]:
import time
total_epochs, total_penalties = 0, 0
episodes = 100
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

In [8]:
with open("weight.coe", 'w') as f:
    f.write("memory_initialization_radix=10;\n")
    f.write("memory_initialization_vector=\n")
    for i in range(3**10):
        f.write(str(np.argmax(q_table[i])))
        if(i == 3**10 - 1):
            f.write(";\n")
        else:
            f.write(",\n")